# Week 3 Introduction to image processing assignment

# importing the required libraries

In [1]:
!pip install pydicom
import pydicom
import numpy as np 
import glob 
import os
import matplotlib.pyplot as plt
import json 
from ipywidgets import interact, interactive
from typing import List, Tuple, Union

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.2 MB/s eta 0:00:00


# downloading the data
#### In this section we are going to download a public dataset and unzip it for later usage. 

In [2]:
! wget --no-check-certificate https://data.idoimaging.com/dicom/1010_brain_mr/1010_brain_mr_04_lee.zip
! unzip 1010_brain_mr_04_lee.zip

--2023-04-14 07:50:08--  https://data.idoimaging.com/dicom/1010_brain_mr/1010_brain_mr_04_lee.zip
Resolving data.idoimaging.com (data.idoimaging.com)... 52.84.52.23, 52.84.52.21, 52.84.52.46, ...
Connecting to data.idoimaging.com (data.idoimaging.com)|52.84.52.23|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 2212367 (2.1M) [application/zip]
Saving to: ‘1010_brain_mr_04_lee.zip’

1010_brain_mr_04_le 100%[===================>]   2.11M  --.-KB/s    in 0.09s   

2023-04-14 07:50:08 (23.3 MB/s) - ‘1010_brain_mr_04_lee.zip’ saved [2212367/2212367]

Archive:  1010_brain_mr_04_lee.zip
   creating: 1010_brain_mr_04_lee/
  inflating: 1010_brain_mr_04_lee/img_000.dcm  
  inflating: 1010_brain_mr_04_lee/img_001.dcm  
  inflating: 1010_brain_mr_04_lee/img_002.dcm  
  inflating: 1010_brain_mr_04_lee/img_003.dcm  
  inflating: 1010_brain_mr_04_lee/img_004.dcm  
  inflating: 1010_brain_mr_04_lee/img_005.dcm  
  inflating: 1010_brain_mr_04_l

In [3]:
dir_path = '1010_brain_mr_04_lee'

# Excercise 1: Read Dicom files from a directory. 
For this question, you need to read all the `.dcm` files inside the provided directory slice by slice, and return the loaded slices in a format of a single `Numpy List` variable. 

* Define a function for reading the DICOM files and sort them based on their `instance numbers`
* Access the patient ID from the dicom object using the following ways and print them: 
  - indexing 
  - attributes (tags)

In [4]:
# solution
def load_dicom_slices(dir_path: str, force: bool=False):
  """ Load and sort a series of dicom files inside the provided folder path. 
  """
  files = os.listdir(dir_path)
  slices = []
  for file in files:
      if file.endswith('.dcm'):
          ds = pydicom.dcmread(os.path.join(dir_path, file))
          slices.append(ds)

  slices.sort(key=lambda x: int(x.InstanceNumber))
  return slices

slices = load_dicom_slices(dir_path)
print("Number of slices: ", len(slices))
print('Slices dtype: ', type(slices[0]))

Number of slices:  32
Slices dtype:  <class 'pydicom.dataset.FileDataset'>


In [ ]:
# Viewing slices shape. 
print("Volume Shape (Row, Column): ", slices[0].Rows, slices[0].Rows)

In [ ]:
# interactive slides for viewing dicom slides
plt.figure(1, figsize=(10, 10))
def dicom_animation(x):
    plt.imshow(slices[x].pixel_array, cmap=plt.cm.bone)
    plt.colorbar()
    return x

interact(dicom_animation, x=(0, len(slices)-1))

In [10]:
# Accessing the patient id. 
# Access the patient ID using indexing:
patient_id = slices[0].PatientID
print("Patient ID: ",patient_id)

# Access the patient ID using attributes (tags):
patient_id = slices[0].get("PatientID")
print("Patient ID: ",patient_id)

Patient ID:  1010
Patient ID:  1010


# Exercise 2: 
#### For this exercise you need to implement four function and try applying them on the loaded slices. 
* Define a function named `to_hu` for transforming the slices into Hounsfield scale. 
* Define a function named `window_clip` for cliping the pixel intensity range of each slice using a single center and windows width. Defined window describes your interested intensity range. 
* Define a function named `to_3d_numpy` to convert all the slices into a single Numpy ndarray image. This function is able to change the datatype of the output image if the user likes to change the new image datatype. 
* Define a function named `min_max_scaler` to scale a Numpy array into range `0` and `1` for easier visualization with matplotlib. This function is able to change the datatype (optional dtype by user) of the output image to `float`. 



In [28]:
def to_hu(slices):
    """
    Transforms a list of DICOM datasets into Hounsfield units (HU).
    :param dicom_datasets: A list of PyDICOM datasets to be transformed.
    :return: A list of transformed datasets as NumPy arrays.
    """
    hu_slices = []
    intercept = slices[0].RescaleIntercept if 'RescaleIntercept' in slices[0] else 0
    slope = slices[0].RescaleSlope if 'RescaleSlope' in slices[0] else 1
    for ds in slices:
        hu = ds.pixel_array * slope + intercept
        hu_slices.append(hu)
    return hu_slices

In [ ]:
def to_hu(slices: List):
    """Transform a list of slices to a Hounsfield Unit Scale. 
    This function takes the loaded slices and return a list of transformed Numpy array format slices. 
    """
    hu_slices = []
    intercept = slices[0].RescaleIntercept if 'RescaleIntercept' in slices[0] else 0
    slope = slices[0].RescaleSlope if 'RescaleSlope' in slices[0] else 1
    for sli in slices: 
      hu_image = sli.pixel_array * slope + intercept
      hu_slices.append(hu_image) 
    return hu_slices

def window_clip(slices: List, window_cent: int, window_width: int):
    """Clip a list of slices pixels, one by one, into a specific intensity range based on the provided window location and size.
    All the pixels inside each single slice with a intensity below and over the window range will be clipped into the min and max intensity range window covers. 
    This function returns a list of clipped Numpy array slices. 
    """
    cliped_slices = []
    for sli in slices:
      clipped = np.clip(sli, window_cent - (window_width/2), window_cent + (window_width/2))
      cliped_slices.append(clipped)
    return cliped_slices
    

def to_3d_numpy(slices: List, dtype=None): 
    """Stack up all slices into a single NumPy array of the provided data type.
    """
    image = np.stack(slices)
    if dtype: 
        image = image.astype(dtype)
    return image

def min_max_scaler(image: np.ndarray, dtype: Union[type, None]=None): 
    """Scale a single Numpy array image intensity into range `0` and `1`
    """
    min_val = np.min(image)
    max_val = np.max(image)
    image = (image - min_val) / (max_val - min_val)
    if dtype:
      image = image.astype(dtype)
    return image

def visualizer(slice: np.ndarray, title= ''): 
    """Visualize a slice of type numpy array with the provided title."""
    plt.imshow(slice, cmap=plt.cm.bone)
    plt.title(title)
    plt.show()

In [ ]:
processed_slices_hu = to_hu(slices)
processed_slices_cliped = window_clip(processed_slices_hu, 500, 1000) # Extracting soft tissues. 
image = to_3d_numpy(processed_slices_cliped, dtype=None)

print('Numpy array image shape is: ', image.shape)
print('Transformed image pixel value range (min, max): ', (image.min(), image.max()))

scaled_image = min_max_scaler(image, dtype=np.float32)
print('Scaled image pixel value range (min, max): ', (scaled_image.min(), scaled_image.max()))

# You can see the scaled version of your slected region using the `window_clip` function here.
visualizer(processed_slices_hu[10], 'HU transform')
visualizer(scaled_image[10], 'Scaled')

# Exercise 3: Saving the data in numpy format
* Convert original frames into a numpy image array without any preprocessing, and save it as a single `.npy` file named `original_numpy_version.npy`. Keeping the original slices as a 3d numpy sample in case we needed it in the future.
* Create an ouput folder named `scaled_slices` and save all the scaled slices one by one into the folder. Assign a unique name to each slice while you are keeping the original order. 

In [ ]:
# Convert the original slices into a 3d nmpy array and save it as a .npy file format. 
def to_3d_numpy(slices, output_file):
    
    # Sort the DICOM files by InstanceNumber
    slices.sort(key=lambda x: pydicom.dcmread(x).InstanceNumber)

    # Read the first DICOM file to get the shape of the pixel data
    first_dicom = pydicom.dcmread(slices[0])
    rows = int(first_dicom.Rows)
    cols = int(first_dicom.Columns)
    slices = len(slices)

    # Create a 3D Numpy array to store the pixel data
    pixel_data = np.zeros((slices, rows, cols), dtype=np.int16)

    # Read the pixel data from each DICOM file and store it in the Numpy array
    for i, file in enumerate(slices):
        ds = pydicom.dcmread(file)
        pixel_data[i, :, :] = ds.pixel_array

    # Save the pixel data as a .npy file
    np.save(output_file, pixel_data)

    return pixel_data

# Convert the DICOM files to a 3D Numpy array and save it as a .npy file
pixel_data = to_3d_numpy(slices, "pixel_data.npy")


In [ ]:
# Create a folder and save the scaled slices one by one. 
# create the output directory if it doesn't exist
if not os.path.exists('scaled_slices'):
    os.makedirs('scaled_slices')

# loop through the DICOM datasets and save them as individual slices
for i, dataset in enumerate(scaled_image):
    ds = pydicom.dataset.Dataset.from_dict({'PixelData': dataset.tobytes()})
    filename = f'slice_{i}.dcm'
    filepath = os.path.join('scaled_slices', filename)
    pydicom.dcmwrite(filepath, ds)


# Exercise 4: Save the processed numpy data as a series of DICOM files
#### In this section, you need to save the scaled version image of the loaded dicom slices into a series of dicom files inside a directory named `scaled_slices_dicom`, be splitting the image into a sequence of slices in the original order.  
To do so, you need to create a new dicom dataset for each slide, import the original dicom information into the new created dataset, and save it with a unique name inside the mentioned directory.

In [ ]:
# create the output directory if it doesn't exist
if not os.path.exists('scaled_slices_dicom'):
    os.makedirs('scaled_slices_dicom')

# read the DICOM series into a list of slices
slices = []
for filename in os.listdir(dir_path):
    filepath = os.path.join(dir_path, filename)
    ds = pydicom.dcmread(filepath)
    slices.append(ds.pixel_array)

# convert the list of slices to a numpy array and scale the pixel values
image = np.array(slices)
scaled_image = (image - np.min(image)) / (np.max(image) - np.min(image)) * 65535
scaled_image = scaled_image.astype(np.uint16)

# loop through the slices and save them as individual DICOM files
for i, slice_data in enumerate(scaled_image):
    ds = pydicom.dataset.Dataset.from_dict({'PixelData': slice_data.tobytes()})
    # set the DICOM attributes here, like the pixel spacing, etc.
    # ...
    filename = f'slice_{i}.dcm'
    filepath = os.path.join('scaled_slices_dicom', filename)
    pydicom.dcmwrite(filepath, ds)
